# 0. Daten vorbereiten

Zunächst werden die Daten aus den einzelnen .txt Dateien ausgelesen und in ein gemeinsames Dataframe gepackt.

In [ ]:
print("Einlesen der ursprünglichen Daten...")

In [ ]:
column_names = ['Timestamp', 'ID', 'X', 'Y']
df = pd.DataFrame(columns = column_names)

# durch jede .txt Datei im Ordner 'Data' iterieren, Daten einlesen und ans Dataframe anhängen
for filename in os.listdir("Data/raw"): 
    if filename.endswith(".txt"):
        tmp_df = pd.read_csv("Data/raw/" + filename, sep=" ", header=None)
        tmp_df.columns = column_names 
        df = df.append(tmp_df, ignore_index = True)

## 0.1 Vereinheitlichen von IDs und Timestamps | Deltas zwischen zwei Zeitpunkten berechnen

Jetzt werden die Daten aufbereitet, damit man besser mit ihnen arbeiten kann. Darunter fällt das **Vergeben einer einzigartigen ID für jeden Datenpunkt** (1 Datenpunkt besteht aus 20 Zeitpunkten), sowie das **Vereinheitlichen der Timestamps**. Da für uns nur die Reihenfolge der einzelnen Zeitpunkte von Interesse ist, der Abstand zwischen diesen auch immer gleich ist, können wir den Zeitpunkten die Werte 1-20 zuordnen (wobei zu Zeitpunkt 1 das erste mal x und y Wert erfasst wurde und bei Zeitpunkt 20 das letzte mal). </br>
Außerdem werden in diesem Schritt für die X- und Y-Positionen die **relativen Änderungen (Deltas) zwischen zwei Zeitpunkten berechnet** und in den separaten Spalten 'dx' und 'dy' abgelegt. Die Delta-Werte bilden im Weiteren Verlauf unsere Features, d. h. das später trainierte neuronale Netz sagt uns zukünftig erwartete Delta-Positionen für eine gewisse Anzahl an gegebenen Delta-Werten voraus, wodurch sich dann wiederum die zukünftigen Positionen bestimmen lassen.

In [ ]:
print("\nVereinheitlichen von IDs und Timestamps und berechnen der Deltas zwischen zwei Zeitpunkten...")

In [ ]:
curr_id = 0
prev_x = 0
prev_y = 0
for i, row in df.iterrows():
    if i % 20 == 0:
        curr_id += 1 # alle 20 Zeitpunkte erhöht sich die ID
        prev_x = row.X
        prev_y = row.Y
    df.at[i, 'ID'] = curr_id
    df.at[i, 'Timestamp'] = (i % 20) + 1
    dx = row.X - prev_x # Delta x berechnen
    dy = row.Y - prev_y # Delta y berechnen
    prev_x = row.X
    prev_y = row.Y
    df.at[i, 'dx'] = round(dx, 3)
    df.at[i, 'dy'] = round(dy, 3)

## 0.2 Aufteilen der Daten in Trainings-, Test- und Validierungsdaten

Anschließend werden die Daten in **Test- (80%), Trainings- und Validierungsdaten (je 10%)** aufgeteilt und jeweils als eigene CSV-Datei abgespeichert. Wurden dieser Split bereits schon einmal durchgeführt, so werden die Daten nicht erneut aufgeteilt, sondern einfach die bereits existierenden CSV-Dateien eingelesen. 

*Anmerkung: Will man das neuronale Netz mit einer neuen Datenaufteilung trainieren, validieren und testen, so müssen einfach die drei CSV-Dateien aus dem Ordner "Data/datasplits/" entfernt werden, damit diese bei erneutem Programmdurchlauf neu erzeugt werden.* 

In [ ]:
print("\nAufteilen der Daten in Trainings- Validations- und Testdaten...")

In [ ]:
ids = np.array(df.ID.unique()) # alle IDs holen
np.random.shuffle(ids) # IDs zufällig durchmischen

test_percentage = valid_percentage = 0.1 # Testdaten und Validierungsdaten je 10%, der Rest sind Trainingsdaten
test_size = int(test_percentage * len(ids))
valid_size = int(valid_percentage * len(ids))
test_ids, valid_ids, train_ids = ids[:test_size], ids[test_size:test_size + valid_size], ids[test_size + valid_size:]

df_train = df.copy()
for id in np.concatenate([test_ids, valid_ids]): # alle Datenpunkte der Test- und Validierungsdaten entfernen --> übrig bleiben die Trainingsdaten
    df_train = df_train.drop(df_train[df_train.ID == id].index) # --> drop- Methode ist schneller als die append Methode
df_test = df.copy()
for id in np.concatenate([train_ids, valid_ids]): # alle Datenpunkte der Trainings- und Validierungsdaten entfernen --> übrig bleiben die Testdaten
    df_test = df_test.drop(df_test[df_test.ID == id].index)
df_valid = df.copy()
for id in np.concatenate([test_ids, train_ids]): # alle Datenpunkte der Test- und Trainingsdaten entfernen --> übrig bleiben die Validierungsdaten
    df_valid = df_valid.drop(df_valid[df_valid.ID == id].index)
df_train.to_csv(path_train, sep=',', index = False)
df_test.to_csv(path_test, sep=',', index = False)
df_valid.to_csv(path_valid, sep=',', index = False)

In [ ]:
print("Trainings- und Testdaten wurden aufgeteilt und sind nun im Ordner 'Data/datasplits' wiederzufinden!")
print("Jetzt kanns losgehen!")